# Sistema de Reconhecimento Facial — Notebook
Este notebook permite coletar faces via upload, treinar o modelo (LBPH ou CNN) e reconhecer faces em uma imagem.

In [ ]:
from pathlib import Path
import os, cv2, numpy as np, pickle
from ipywidgets import FileUpload, Text, Button, Dropdown, VBox, HBox
from IPython.display import display
from face_recognition_app import ensure_dirs, detect_faces, train_recognizer
ensure_dirs()
USE_TF_DETECTOR = os.environ.get('FACE_USE_TF_DETECTOR', '0').lower() in ('1','true','yes')
print('Detecção por rede (MTCNN):', USE_TF_DETECTOR)

## Coleta por upload
Carregue imagens e salve faces recortadas em `dataset/<nome>/`.

In [ ]:
name = Text(description='Nome')
u = FileUpload(accept='.jpg,.jpeg,.png', multiple=True)
btn = Button(description='Salvar faces')
box = VBox([name, u, btn])
def on_click(_):
    if not name.value:
        print('Informe um nome'); return
    person_dir = Path('dataset')/name.value
    person_dir.mkdir(parents=True, exist_ok=True)
    count = 0
    for item in u.value.values():
        data = np.frombuffer(item['content'], dtype=np.uint8)
        img = cv2.imdecode(data, cv2.IMREAD_COLOR)
        if img is None: continue
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = detect_faces(gray)
        for (x,y,w,h) in faces:
            face = cv2.resize(gray[y:y+h, x:x+w], (200,200))
            next_i = len(list(person_dir.glob('*.jpg')))
            cv2.imwrite(str(person_dir/f'{next_i:03d}.jpg'), face)
            count += 1
    print(f'Faces salvas: {count} em {person_dir}')
btn.on_click(on_click)
display(box)

## Treinamento
Escolha o método e treine o modelo.

In [ ]:
method = Dropdown(description='Método', options=['lbph','cnn'], value='lbph')
train_btn = Button(description='Treinar')
train_box = HBox([method, train_btn])
def train_click(_):
    ok = train_recognizer(method.value)
    print('Treinado:', ok)
train_btn.on_click(train_click)
display(train_box)

## Predição por upload
Envie uma imagem para reconhecer e visualize o resultado.

In [ ]:
from tensorflow import keras
import matplotlib.pyplot as plt
labels_path = Path('trainer')/'labels.pickle'
model_tf_path = Path('trainer')/'model_tf.h5'
model_lbph_path = Path('trainer')/'trainer.yml'
upload_pred = FileUpload(accept='.jpg,.jpeg,.png', multiple=False)
pred_btn = Button(description='Reconhecer')
display(VBox([upload_pred, pred_btn]))
def predict_upload(_):
    if not labels_path.exists() or (not model_tf_path.exists() and not model_lbph_path.exists()):
        print('Modelo não encontrado'); return
    if not upload_pred.value:
        print('Envie uma imagem'); return
    with open(labels_path, 'rb') as f:
        label_ids = pickle.load(f)
    id_labels = {v:k for k,v in label_ids.items()}
    use_tf = model_tf_path.exists()
    if use_tf:
        model = keras.models.load_model(str(model_tf_path))
        recognizer = None
    else:
        recognizer = cv2.face.LBPHFaceRecognizer_create()
        recognizer.read(str(model_lbph_path))
    item = list(upload_pred.value.values())[0]
    data = np.frombuffer(item['content'], dtype=np.uint8)
    image = cv2.imdecode(data, cv2.IMREAD_COLOR)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detect_faces(gray)
    for (x,y,w,h) in faces:
        face = cv2.resize(gray[y:y+h, x:x+w], (200,200))
        if use_tf:
            arr = (face.astype('float32')/255.0)[None,...,None]
            probs = model.predict(arr, verbose=0)[0]
            lid = int(np.argmax(probs)); conf = float(np.max(probs)*100.0)
            name = id_labels.get(lid,'desconhecido')
            text = f'{name} ({conf:.1f}%)'
        else:
            lid, conf = recognizer.predict(face)
            name = id_labels.get(lid,'desconhecido')
            text = f'{name} ({conf:.1f})'
        cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
        cv2.putText(image,text,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,0),2)
    bgr = image
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(8,6)); plt.imshow(rgb); plt.axis('off'); plt.show()
pred_btn.on_click(predict_upload)